<a href="https://colab.research.google.com/github/hrbolek/learning/blob/master/ais/05_dataanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analýza a vizualizace dat

- Analýza a vizualizace dat 
    - Nástroje pro platformu Python
    - Pandas
    - Matplotlib
    - Další nástroje (numpy, scikit, ...)

Nástroje pro platformu Python

## Pandas

## Matplotlib

## Další nástroje